# **任务二 多层感知机 Multilayer Perceptron (MLP)**

在任务一中我记录了关于线性层的用法，并借助线性回归简要说明了Torch框架的训练流程，这次任务中正式记录深度学习的第一种经典模型——多层感知机

___

## 1. 任务说明

多层感知机本质上就是一个很多层线性层的多层神经网络，只不过为了拟合更复杂的任务模型，需要引入一些新的结构

本次任务中着重说明这些内容，并展示多层感知机

## 2. 模型定义

In [150]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.Linear_layer0 = nn.Linear(input_dim, hidden_dim)
        self.Linear_layer1 = nn.Linear(hidden_dim, hidden_dim)
        self.Linear_layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.Linear_layer3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.Linear_layer0(x)
        x = self.relu(x)
        x = self.Linear_layer1(x)
        x = self.relu(x)
        x = self.Linear_layer2(x)
        x = self.relu(x)
        x = self.Linear_layer3(x)
        x = self.relu(x)
        return x

就代码而言，我们能看到使用了更多空间来定义，我们的模型结构，这就是增加模型复杂度的过程。

这次模型除了上次用到的 `input_dim`, `output_dim` 还引入的 `hidden_dim`。我们知道线性层会将 `dim` 维度从 `input_dim` 变为 `output_dim`，所以直接将 `nn.Linear(input_dim, output_dim)`简单堆叠是不行的，因为第一层的输出维度为 `output_dim`，与第二层需要的 `1input_dim`出现差异，除非你的输入输出维度完全相同，所以我们这里使用了 `hidden_dim` 作为中间维度，也就是常说的隐藏层维度。

我认为将多层感知机结构划分成：输入层、隐藏层、输出层，是不利于理解的，不如直接从维度变化的角度考虑为什么需要做出这些区分。

除了层数的变化之外，还引入了 `nn.ReLU()`, 它的存在是为了利用一个非线性公式增加模型在复杂任务的处理能力，被称为**激活函数**，这类函数的种类也有很多，这里依然不做强调。

## 3. 数据生成

这里先随机的，有意识的生成一些存在较明显规则边界的数据，主要为了展示模型的训练过程。

| 数据类型	   | 特征 / 目标	     | 取值范围	                    | 生成逻辑（核心公式）                 |
|---------|--------------|--------------------------|----------------------------|
| 输入特征（X） | 	设定温度（T_set） | 	16 ~ 30 ℃	              | 均匀随机生成（空调常用设定范围）           |
| -       | 环境温度（T_env）  | 	20 ~ 38 ℃	              | 均匀随机生成（夏季室内常见温度）           |
| -       | 运行模式（mode）   | 	0（制冷）、1（制热）、2（送风）       | 	随机采样（三类模式等概率）             |
| -       | 运行时长占比	      | 0 ~ 1（0 = 未开，1 = 满小时运行）	 | 均匀随机生成                     |
| -       | 室内人数         | 	1 ~ 5 人	                | 整数随机生成（家庭常见人数）             |
| 目标变量（y） | 每小时耗电量	      | 0.1 ~ 2.5 kWh	           | 基础能耗 + 温度差影响 + 人数影响 + 随机噪声 |


In [151]:
def get_data(data_size):
    # 2. 生成输入特征 [batch_size, 5]（5个特征）
    T_set = torch.rand(data_size, 1) * 14 + 16  # 16~30℃
    T_env = torch.rand(data_size, 1) * 18 + 20  # 20~38℃
    mode = torch.randint(0, 3, (data_size, 1))  # 0/1/2（运行模式）
    time_ratio = torch.rand(data_size, 1)       # 0~1（运行时长占比）
    people = torch.randint(1, 6, (data_size, 1))# 1~5人
    X = torch.cat([T_set, T_env, mode, time_ratio, people], dim=1)  # 拼接为[batch,5]

    # 3. 生成目标变量（耗电量）[batch_size, 1]
    base_power = 0.2  # 基础能耗
    # 温度差影响（分模式）
    temp_effect = torch.where(mode == 0, 0.08 * torch.max(T_env - T_set, torch.zeros_like(T_env)) * time_ratio,
                              torch.where(mode == 1, 0.1 * torch.max(T_set - T_env, torch.zeros_like(T_env)) * time_ratio,
                                          0.01 * time_ratio))
    # 人数影响
    people_effect = 0.05 * people * time_ratio
    # 随机噪声（-0.1~0.1）
    noise = (torch.rand(data_size, 1) - 0.5) * 0.1
    # 最终耗电量
    y = base_power + temp_effect + people_effect + noise

    # 最终数据形状：X→[128,5]（输入），y→[128,1]（目标），适配MLP输入格式
    return X, y

# 划分批次
def split_batch(data, batch_size):
    # 核心操作：沿第一个维度（dim=0）分割，保留后续所有维度
    split_tensors = torch.split(data, batch_size, dim=0)
    # 转为列表返回（torch.split返回tuple，列表更易操作）
    return list(split_tensors)

# 训练数据
train_x, train_y = get_data(1024)
train_x_batch = split_batch(train_x, 128)
train_y_batch = split_batch(train_y, 128)
# 验证数据
val_x, val_y = get_data(128)
# 测试数据
test_x, test_y = get_data(8)
print('输入数据形状:', train_x.shape)
print('输入批次数量:', len(train_x_batch), '\t批次形状:', train_x_batch[0].shape)
print('标签数据形状:', train_y.shape)
print('输入批次数量:', len(train_y_batch), '\t批次形状:', train_y_batch[0].shape)


输入数据形状: torch.Size([1024, 5])
输入批次数量: 8 	批次形状: torch.Size([128, 5])
标签数据形状: torch.Size([1024, 1])
输入批次数量: 8 	批次形状: torch.Size([128, 1])


本次数据生成除了训练数据还生成了测试数据，这项工作实际上在线性回归也进行了，只不过由于线性回归的数据比较简单，是在模型训练结束后直接在测试是生成的.

验证数据则是为了训练过程中检测模型的训练状态，主要是为了观察”过拟合“情况，通常表现为损失函数下降，验证准确率缺在降低（或误差率增加）。

在先前任务一中，由于数据量并不大，所以没有划分批次大小，实际上咱们先前的batch_size就是现在的data_size，此次任务中我们将data_size划分成若干的batch_size的张量，用列表容纳。

## 3. 模型训练

### 3.1 实例化模型、损失函数、优化器

本次任务仍然是回归任务，使用均方误差 `nn.MSELoss()`。

In [152]:
# 线性回归的输入和输出均只有一个维度
model = Model(5, 1, 128)
# 定义损失函数，这里使用均方误差损失（MSELoss）
criterion = nn.MSELoss()
# 定义优化器，使用随机梯度下降（SGD）来更新权重和偏置
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### 3.2 **迭代训练**

In [153]:
epochs = 1000
for epoch in range(1000):
    loss = None
    model.train()
    for i in range(len(train_x_batch)):
        x = train_x_batch[i]
        y = train_y_batch[i]

        # 前向传播，得到预测值
        output = model(x)
        # 计算损失
        loss = criterion(output, y)
        # 梯度清零，因为在每次反向传播前都要清除之前累积的梯度
        optimizer.zero_grad()
        # 反向传播，计算梯度
        loss.backward()
        # 更新权重和偏置
        optimizer.step()

    model.eval()
    output = model(val_x)
    val_loss = criterion(output, val_y).item()
    error = torch.sum((val_y - output) ** 2).item()
    if (epoch + 1) % 200 == 0:
        print(f'[epoch {epoch+1}]loss:', loss.item())
        print(f'\t val loss:', val_loss)
        print(f'\t val error:', error)


[epoch 200]loss: 0.02839551866054535
	 val loss: 0.05407831817865372
	 val error: 6.922024726867676
[epoch 400]loss: 0.025329984724521637
	 val loss: 0.04664250463247299
	 val error: 5.970240592956543
[epoch 600]loss: 0.023334629833698273
	 val loss: 0.04175221174955368
	 val error: 5.344283103942871
[epoch 800]loss: 0.02162093110382557
	 val loss: 0.038202136754989624
	 val error: 4.889873504638672
[epoch 1000]loss: 0.020201371982693672
	 val loss: 0.03547819331288338
	 val error: 4.541208744049072


在本次任务中，尽管任务足够简单，但对于模型性能的把控仍然需要不断测试，包含学习率的调整，隐藏层大小的设置

实际训练中就遇到以下几种情况

损失不降低：
- 模型结构过于简单，无法成功拟合数据；
- 因为学习率过高，导致模型在两个损失较高的”山腰“反复跳跃而不能走向”山谷“

模型损失值正在下降，验证性能却很差（本次任务中表象是误差值较高）：
- 模型陷入过拟合，内在原因是模型过于复杂，在训练集表现力过强导致无法泛化

除此之外，数据有明确规则边界而随机批量生成的数据，强行堆数据量可能出现训练样本与验证集出现重叠，导致验证结果可能不能体现训练性能。


### 3.3 **测试模型**

使用 `model.eval()` 将模型改为测试模式，避免自动的梯度计算增加额外的计算量。

In [154]:
model.eval()
output = model(test_x)

for i in range(len(test_x)):
    print('输入数据:', test_x[i].tolist())
    print('目标结果:', test_y[i].item())
    print('预测结果:', output[i].item())

输入数据: [27.860410690307617, 28.998764038085938, 0.0, 0.858627200126648, 4.0]
目标结果: 0.41962671279907227
预测结果: 0.4485475420951843
输入数据: [17.162025451660156, 34.69712829589844, 2.0, 0.8450523614883423, 1.0]
目标结果: 0.22942206263542175
预测结果: 0.26342248916625977
输入数据: [27.040096282958984, 36.28170394897461, 1.0, 0.6865988969802856, 4.0]
目标结果: 0.34881022572517395
预测结果: 0.43733468651771545
输入数据: [20.2906494140625, 27.017436981201172, 0.0, 0.41324108839035034, 4.0]
目标结果: 0.4932757318019867
预测结果: 0.46766433119773865
输入数据: [22.358427047729492, 34.11265563964844, 0.0, 0.1415175199508667, 1.0]
目标结果: 0.3388589024543762
预测结果: 0.454328328371048
输入数据: [28.299440383911133, 26.190929412841797, 0.0, 0.41163206100463867, 3.0]
目标结果: 0.26485496759414673
预测结果: 0.3284117877483368
输入数据: [26.3489990234375, 20.084047317504883, 1.0, 0.9513846039772034, 1.0]
目标结果: 0.8338932991027832
预测结果: 0.42180994153022766
输入数据: [19.71518898010254, 25.95184898376465, 1.0, 0.04220020771026611, 5.0]
目标结果: 0.24780318140983582
预测结果: 0.

## 4. **总结**

这是第二个关于 `torch` 框架的任务，也是第一个真正的深度学习任务，第一次引入了激活函数和批次的概念，这些内容对后续更复杂的任务有很大帮助。